In [5]:
# import necessary libraries
import requests
import pandas
from bs4 import BeautifulSoup
import re

In [28]:
# define data containers
book_title = []
author = []
published = []
n_ratings = []
avg_rating = []
editions = []
for page in range(1,6):
    url=f"https://www.goodreads.com/search?page={page}&q=data+science&qid=1S5Po1JzOT&search_type=books&tab=books&utf8=✓"
    response=requests.get(url)
    soup=BeautifulSoup(response.content)
    table=soup.find("table")
    #finding rows
    books = table.find_all('tr')
    for book in books:
        #finding titles
        title = book.find_all('a', class_ = 'bookTitle')
        book_title.append(title[0].text.strip())
        #find published year
        pattern = re.compile(r"published\s*(\d{4})")
        year_pub = pattern.search(book.text)
        published.append(year_pub.group(1) if year_pub else 0)
        #finding ratings
        all_ratings = book.find_all('span', class_ = 'minirating')
        all_ratings_text = all_ratings[0].text.strip()
        pattern_2 = re.compile(r"(\d\.?\d*)\savg")
        avg_rating.append(pattern_2.search(all_ratings_text).group(1))
        #total number of ratings
        pattern_4 = re.compile(r"(\d\,?\d*) rating")
        ratings_matches = pattern_4.search(all_ratings_text)
        n_ratings.append(ratings_matches.group(1) if ratings_matches else 0)  
        #editions
        pattern_3 = re.compile(r"(\d+) edition")
        eds = book.find(href=re.compile("editions"))
        editions.append(pattern_3.search(eds.text).group(1))
        #finding authors
        authors_for_book = []
        authors_names = book.find_all('a', class_ = 'authorName')
        for authors in authors_names:   
            authors_for_book.append(authors.text.strip())
        author.append(authors_for_book)

df = pd.DataFrame({
    'Title': book_title, 
    'Author(s)': author,
    'Published Year': published,
    'Editions': editions,
    'Avg. Rating': avg_rating,
    'Num. of Ratings': n_ratings})


In [30]:
df.head()

,Title,Author(s),Published Year,Editions,Avg. Rating,Num. of Ratings
0,Data Science for Business: What you need to kn...,"[Foster Provost, Tom Fawcett]",2013,14,4.16,"1,696"
1,Data Smart: Using Data Science to Transform In...,[John W. Foreman],2013,15,4.14,877
2,Data Science from Scratch: First Principles wi...,[Joel Grus],2015,20,3.93,722
3,Python Data Science Handbook: Tools and Techni...,[Jake Vanderplas],2016,8,4.32,315
4,Doing Data Science,"[Rachel Schutt, Cathy O'Neil]",2013,9,3.71,501
